In [ ]:
#instlall the libraries
# pip install mapclassify
# pip install fiona
# pip install shapely
# pip install pyproj
# pip install rtree
# pip install geopandas
# pip install openpyxl
# pip install leafmap
# pip install geopy
# pip install geemap
# pip install folium
# pip install zipfile
# pip install pandas
# pip install numpy
# pip install matplotlib

In [1]:
#importing the necessary libraries
import os
import re
import folium
import matplotlib
import mapclassify
import pandas as pd
import geopandas as gpd
import leafmap.foliumap as leafmap
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from zipfile import ZipFile

In [2]:
#setting the working directory
data_folder = 'data'
output_folder = 'output'

if not os.path.exists(data_folder):
    os.mkdir(data_folder)
if not os.path.exists(output_folder):
    os.mkdir(output_folder)

In [7]:
os.getcwd()

'C:\\Users\\bdhungan\\Downloads\\geocoding'

In [8]:
#reading the excel file
excel_file = r"C:\Users\bdhungan\Downloads\geocoding\data\data\TWDB_Active_Policies_Subset.xlsx"
excel_file_path = os.path.join(data_folder, excel_file)
address_df = pd.read_excel(excel_file_path)
address_df

,Community Name (Number),Address Line 1,Address Line 2,City,State,Zip Code,Zip Plus 4
0,"ABILENE, CITY OF (485450)",1001 ELMWOOD DR,NaN,ABILENE,TX,79605,NaN
1,"ABILENE, CITY OF (485450)",1001 MATADOR ST,NaN,ABILENE,TX,79605,3019.0
2,"ABILENE, CITY OF (485450)",10257 APACHE LN,NaN,ABILENE,TX,79601,8286.0
3,"ABILENE, CITY OF (485450)",1034 WESTVIEW DR,NaN,ABILENE,TX,79603,4722.0
4,"ABILENE, CITY OF (485450)",109 TIQUEWOOD CIR,NaN,ABILENE,TX,79605,4937.0
...,...,...,...,...,...,...,...
94,"ABILENE, CITY OF (485450)",325 MOORE DR,NaN,ABILENE,TX,79605,1619.0
95,"ABILENE, CITY OF (485450)",3266 COLLEGE ST,NaN,ABILENE,TX,79605,4041.0
96,"ABILENE, CITY OF (485450)",3301 GREEN ACRES RD,NaN,ABILENE,TX,79605,5030.0
97,"ABILENE, CITY OF (485450)",3309 PARKCREST DR,NaN,ABILENE,TX,79605,5024.0


In [7]:
# formating the address if not in standard ordinal form
# def make_ordinal(match):
#     n = int(match.group(1))
#     if 11 <= (n % 100) <= 13:
#         suffix = 'th'
#     else:
#         suffix = ['th', 'st', 'nd', 'rd', 'th'][min(n % 10, 4)]
#     return str(n) + suffix + match.group(2)

# def update_address(row):
#   old_address = row['Address Line 1']
#   pattern = r'(\d+)(\s+(?:ST|AV|RD|DR|PKWY|CIR|LN|AVE))'
#   result = re.sub(pattern, make_ordinal, old_address)
#   return result

# address_df['ADDRESS_FIXED'] = address_df.apply(update_address, axis=1)
# address_df

In [9]:
#create a new column with the full address
address_df['Full_Address'] = (
    address_df['Address Line 1'] + ',' +
    address_df['City'] + ',' +
    address_df['State']+ ',' +
    address_df['Zip Code'].astype(str))
address_df

,Community Name (Number),Address Line 1,Address Line 2,City,State,Zip Code,Zip Plus 4,Full_Address
0,"ABILENE, CITY OF (485450)",1001 ELMWOOD DR,NaN,ABILENE,TX,79605,NaN,"1001 ELMWOOD DR,ABILENE,TX,79605"
1,"ABILENE, CITY OF (485450)",1001 MATADOR ST,NaN,ABILENE,TX,79605,3019.0,"1001 MATADOR ST,ABILENE,TX,79605"
2,"ABILENE, CITY OF (485450)",10257 APACHE LN,NaN,ABILENE,TX,79601,8286.0,"10257 APACHE LN,ABILENE,TX,79601"
3,"ABILENE, CITY OF (485450)",1034 WESTVIEW DR,NaN,ABILENE,TX,79603,4722.0,"1034 WESTVIEW DR,ABILENE,TX,79603"
4,"ABILENE, CITY OF (485450)",109 TIQUEWOOD CIR,NaN,ABILENE,TX,79605,4937.0,"109 TIQUEWOOD CIR,ABILENE,TX,79605"
...,...,...,...,...,...,...,...,...
94,"ABILENE, CITY OF (485450)",325 MOORE DR,NaN,ABILENE,TX,79605,1619.0,"325 MOORE DR,ABILENE,TX,79605"
95,"ABILENE, CITY OF (485450)",3266 COLLEGE ST,NaN,ABILENE,TX,79605,4041.0,"3266 COLLEGE ST,ABILENE,TX,79605"
96,"ABILENE, CITY OF (485450)",3301 GREEN ACRES RD,NaN,ABILENE,TX,79605,5030.0,"3301 GREEN ACRES RD,ABILENE,TX,79605"
97,"ABILENE, CITY OF (485450)",3309 PARKCREST DR,NaN,ABILENE,TX,79605,5024.0,"3309 PARKCREST DR,ABILENE,TX,79605"


In [10]:
#run the geolocator
from tqdm.notebook import tqdm

tqdm.pandas()

locator = Nominatim(user_agent='BDhungana', timeout=10)
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)

address_df_pd = address_df.copy()
address_df_pd['location'] = address_df_pd['Full_Address'].progress_apply(geocode)
address_df_pd

  0%|          | 0/99 [00:00<?, ?it/s]

,Community Name (Number),Address Line 1,Address Line 2,City,State,Zip Code,Zip Plus 4,Full_Address,location
0,"ABILENE, CITY OF (485450)",1001 ELMWOOD DR,NaN,ABILENE,TX,79605,NaN,"1001 ELMWOOD DR,ABILENE,TX,79605","(1001, Elmwood Drive, Abilene, Taylor County, ..."
1,"ABILENE, CITY OF (485450)",1001 MATADOR ST,NaN,ABILENE,TX,79605,3019.0,"1001 MATADOR ST,ABILENE,TX,79605","(1001, Matador Street, Abilene, Taylor County,..."
2,"ABILENE, CITY OF (485450)",10257 APACHE LN,NaN,ABILENE,TX,79601,8286.0,"10257 APACHE LN,ABILENE,TX,79601","(10257, Apache Lane, Abilene, Jones County, Te..."
3,"ABILENE, CITY OF (485450)",1034 WESTVIEW DR,NaN,ABILENE,TX,79603,4722.0,"1034 WESTVIEW DR,ABILENE,TX,79603","(1034, Westview Drive, Abilene, Taylor County,..."
4,"ABILENE, CITY OF (485450)",109 TIQUEWOOD CIR,NaN,ABILENE,TX,79605,4937.0,"109 TIQUEWOOD CIR,ABILENE,TX,79605","(109, Tiquewood Circle, Abilene, Taylor County..."
...,...,...,...,...,...,...,...,...,...
94,"ABILENE, CITY OF (485450)",325 MOORE DR,NaN,ABILENE,TX,79605,1619.0,"325 MOORE DR,ABILENE,TX,79605","(325, Moore Drive, Abilene, Taylor County, Tex..."
95,"ABILENE, CITY OF (485450)",3266 COLLEGE ST,NaN,ABILENE,TX,79605,4041.0,"3266 COLLEGE ST,ABILENE,TX,79605","(3266, College Street, Abilene, Taylor County,..."
96,"ABILENE, CITY OF (485450)",3301 GREEN ACRES RD,NaN,ABILENE,TX,79605,5030.0,"3301 GREEN ACRES RD,ABILENE,TX,79605","(3301, Green Acres Road, Abilene, Taylor Count..."
97,"ABILENE, CITY OF (485450)",3309 PARKCREST DR,NaN,ABILENE,TX,79605,5024.0,"3309 PARKCREST DR,ABILENE,TX,79605","(3309, Parkcrest Drive, Abilene, Taylor County..."


In [11]:
#add latitude and longitude to the dataframe
address_df_pd['latitude'] = address_df_pd['location'].apply(lambda loc: loc.latitude if loc else None)
address_df_pd['longitude'] = address_df_pd['location'].apply(lambda loc: loc.longitude if loc else None)
address_df_pd

,Community Name (Number),Address Line 1,Address Line 2,City,State,Zip Code,Zip Plus 4,Full_Address,location,latitude,longitude
0,"ABILENE, CITY OF (485450)",1001 ELMWOOD DR,NaN,ABILENE,TX,79605,NaN,"1001 ELMWOOD DR,ABILENE,TX,79605","(1001, Elmwood Drive, Abilene, Taylor County, ...",32.438368,-99.768215
1,"ABILENE, CITY OF (485450)",1001 MATADOR ST,NaN,ABILENE,TX,79605,3019.0,"1001 MATADOR ST,ABILENE,TX,79605","(1001, Matador Street, Abilene, Taylor County,...",32.438396,-99.756995
2,"ABILENE, CITY OF (485450)",10257 APACHE LN,NaN,ABILENE,TX,79601,8286.0,"10257 APACHE LN,ABILENE,TX,79601","(10257, Apache Lane, Abilene, Jones County, Te...",32.596423,-99.698836
3,"ABILENE, CITY OF (485450)",1034 WESTVIEW DR,NaN,ABILENE,TX,79603,4722.0,"1034 WESTVIEW DR,ABILENE,TX,79603","(1034, Westview Drive, Abilene, Taylor County,...",32.461541,-99.761730
4,"ABILENE, CITY OF (485450)",109 TIQUEWOOD CIR,NaN,ABILENE,TX,79605,4937.0,"109 TIQUEWOOD CIR,ABILENE,TX,79605","(109, Tiquewood Circle, Abilene, Taylor County...",32.429448,-99.768803
...,...,...,...,...,...,...,...,...,...,...,...
94,"ABILENE, CITY OF (485450)",325 MOORE DR,NaN,ABILENE,TX,79605,1619.0,"325 MOORE DR,ABILENE,TX,79605","(325, Moore Drive, Abilene, Taylor County, Tex...",32.447373,-99.771750
95,"ABILENE, CITY OF (485450)",3266 COLLEGE ST,NaN,ABILENE,TX,79605,4041.0,"3266 COLLEGE ST,ABILENE,TX,79605","(3266, College Street, Abilene, Taylor County,...",32.433266,-99.763239
96,"ABILENE, CITY OF (485450)",3301 GREEN ACRES RD,NaN,ABILENE,TX,79605,5030.0,"3301 GREEN ACRES RD,ABILENE,TX,79605","(3301, Green Acres Road, Abilene, Taylor Count...",32.429782,-99.763491
97,"ABILENE, CITY OF (485450)",3309 PARKCREST DR,NaN,ABILENE,TX,79605,5024.0,"3309 PARKCREST DR,ABILENE,TX,79605","(3309, Parkcrest Drive, Abilene, Taylor County...",32.427916,-99.763816


In [12]:
#find out the failed addresses
failed = address_df_pd[address_df_pd['location'].isna()]
failed

,Community Name (Number),Address Line 1,Address Line 2,City,State,Zip Code,Zip Plus 4,Full_Address,location,latitude,longitude
30,"ABILENE, CITY OF (485450)",1726 SANDPIPER ST,NaN,ABILENE,TX,79602,5208.0,"1726 SANDPIPER ST,ABILENE,TX,79602",None,NaN,NaN
37,"ABILENE, CITY OF (485450)",1917 SOUTHRIDGE XING,NaN,ABILENE,TX,79606,1198.0,"1917 SOUTHRIDGE XING,ABILENE,TX,79606",None,NaN,NaN
59,"ABILENE, CITY OF (485450)",2410 BUTTON WILLOW PKWY,NaN,ABILENE,TX,79606,3510.0,"2410 BUTTON WILLOW PKWY,ABILENE,TX,79606",None,NaN,NaN
73,"ABILENE, CITY OF (485450)",2809 BUTTON WILLOW PKWY,NaN,ABILENE,TX,79606,3517.0,"2809 BUTTON WILLOW PKWY,ABILENE,TX,79606",None,NaN,NaN


In [13]:
#add the address manually
# address_df_pd.loc[54, ['latitude', 'longitude']] = (40.85339,-73.93350)

In [14]:
#rename the columns if needed
# address_df_pd = address_df_pd[['EC_Name', 'Full_Address', 'latitude', 'longitude']]
# address_df_pd.rename(columns = {'EC_Name': 'Name', 'Full_Address': 'Address'}, inplace=True)

In [24]:
#add the geometry column with in the suitable reference system
geometry = gpd.points_from_xy(address_df_pd.longitude, address_df_pd.latitude)
address_gdf = gpd.GeoDataFrame(address_df_pd, crs='EPSG:4326', geometry=geometry)
address_gdf
gdf_local = address_gdf.to_crs('EPSG:2272')
gdf_local

,Community Name (Number),Address Line 1,Address Line 2,City,State,Zip Code,Zip Plus 4,Full_Address,location,latitude,longitude,geometry
0,"ABILENE, CITY OF (485450)",1001 ELMWOOD DR,NaN,ABILENE,TX,79605,NaN,"1001 ELMWOOD DR,ABILENE,TX,79605","(1001, Elmwood Drive, Abilene, Taylor County, ...",32.438368,-99.768215,POINT (-4817931.644 -1668682.709)
1,"ABILENE, CITY OF (485450)",1001 MATADOR ST,NaN,ABILENE,TX,79605,3019.0,"1001 MATADOR ST,ABILENE,TX,79605","(1001, Matador Street, Abilene, Taylor County,...",32.438396,-99.756995,POINT (-4814542.792 -1669534.748)
2,"ABILENE, CITY OF (485450)",10257 APACHE LN,NaN,ABILENE,TX,79601,8286.0,"10257 APACHE LN,ABILENE,TX,79601","(10257, Apache Lane, Abilene, Jones County, Te...",32.596423,-99.698836,POINT (-4782714.322 -1617753.454)
3,"ABILENE, CITY OF (485450)",1034 WESTVIEW DR,NaN,ABILENE,TX,79603,4722.0,"1034 WESTVIEW DR,ABILENE,TX,79603","(1034, Westview Drive, Abilene, Taylor County,...",32.461541,-99.761730,POINT (-4813875.329 -1660933.965)
4,"ABILENE, CITY OF (485450)",109 TIQUEWOOD CIR,NaN,ABILENE,TX,79605,4937.0,"109 TIQUEWOOD CIR,ABILENE,TX,79605","(109, Tiquewood Circle, Abilene, Taylor County...",32.429448,-99.768803,POINT (-4818917.375 -1671812.147)
...,...,...,...,...,...,...,...,...,...,...,...,...
94,"ABILENE, CITY OF (485450)",325 MOORE DR,NaN,ABILENE,TX,79605,1619.0,"325 MOORE DR,ABILENE,TX,79605","(325, Moore Drive, Abilene, Taylor County, Tex...",32.447373,-99.771750,POINT (-4818182.418 -1665206.247)
95,"ABILENE, CITY OF (485450)",3266 COLLEGE ST,NaN,ABILENE,TX,79605,4041.0,"3266 COLLEGE ST,ABILENE,TX,79605","(3266, College Street, Abilene, Taylor County,...",32.433266,-99.763239,POINT (-4816892.261 -1670880.937)
96,"ABILENE, CITY OF (485450)",3301 GREEN ACRES RD,NaN,ABILENE,TX,79605,5030.0,"3301 GREEN ACRES RD,ABILENE,TX,79605","(3301, Green Acres Road, Abilene, Taylor Count...",32.429782,-99.763491,POINT (-4817283.801 -1672101.474)
97,"ABILENE, CITY OF (485450)",3309 PARKCREST DR,NaN,ABILENE,TX,79605,5024.0,"3309 PARKCREST DR,ABILENE,TX,79605","(3309, Parkcrest Drive, Abilene, Taylor County...",32.427916,-99.763816,POINT (-4817550.987 -1672740.684)


In [25]:
#plot the geocoded points in leafmap
m = leafmap.Map(width=800, height=500)
gdf_local.explore(
    m=m,
    marker_type='marker',
    marker_kwds={
        'icon': folium.Icon(color='#fdbb84', icon='hurricane', prefix='fa')
    }
)
m.zoom_to_gdf(address_gdf)
m

In [ ]:
#export the geodataframe to a shapefile
output_file = 'hurricane_evacuation_centers.shp'
output_path = os.path.join(output_folder, output_file)

address_gdf.to_file(filename=output_path)

In [ ]:
#zip the shapefile
output_zip_file = 'hurricane_evacuation_centers.zip'
output_zip_path = os.path.join(output_folder, output_zip_file)

sidecar_files = [
    os.path.join(output_folder, file)
    for file in os.listdir(output_folder)
    if file.endswith(('shp', 'shx', 'dbf', 'prj'))
]

with ZipFile(output_zip_path, 'w') as zip_object:
    for sidecar in sidecar_files:
      zip_object.write(sidecar, os.path.basename(sidecar))

# Optional way to geocode using GeoPandas

In [ ]:
#run the geolocator
locator = Nominatim(user_agent='BDhungan', timeout=10)
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)

address_df_gpd = address_df.copy()

gdf = gpd.tools.geocode(address_df_gpd['ADDRESS_FIXED'])
gdf

In [19]:
#merge the dataframes to get the address
merged = gdf.join(address_df_gpd)
merged

NameError: name 'gdf' is not defined